### Atis example

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys

sys.path.append("../")

warnings.filterwarnings("ignore")

Download atis dataset from [here](https://github.com/Microsoft/CNTK/tree/master/Examples/LanguageUnderstanding/ATIS/Data)

### Run NER model

In [4]:
import os


data_path = "/datadrive/JointSLU/data/"
train_path = os.path.join(data_path, "train_filtered.csv")
valid_path = os.path.join(data_path, "valid_filtered.csv")
model_dir = "/datadrive/models/multi_cased_L-12_H-768_A-12/"
init_checkpoint_pt = os.path.join("/datadrive/models/multi_cased_L-12_H-768_A-12/", "pytorch_model.bin")
bert_config_file = os.path.join("/datadrive/bert/multi_cased_L-12_H-768_A-12/", "bert_config.json")
vocab_file = os.path.join("/datadrive/bert/multi_cased_L-12_H-768_A-12/", "vocab.txt")

In [5]:
import torch
torch.cuda.set_device(0)
torch.cuda.is_available(), torch.cuda.current_device()

(True, 0)

#### Create data loaders

In [6]:
from modules import BertNerData as NerData

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [7]:
data = NerData.create(train_path, valid_path, vocab_file, data_type="bert_cased")

In [8]:
len(data.train_dl.dataset), len(data.valid_dl.dataset)

(9445, 888)

In [9]:
len(data.label2idx) #, len(data.cls2idx)

154

#### Create Ner model

Set params of encoder and decoder as proposed [here](https://arxiv.org/pdf/1609.01454.pdf)

In [10]:
from modules.models.bert_models import BertBiLSTMAttnCRF

In [18]:
model = BertBiLSTMAttnCRF.create(len(data.label2idx), bert_config_file, init_checkpoint_pt, enc_hidden_dim=256)

#### Create learner

In [19]:
from modules import NerLearner

In [20]:
num_epochs = 250
learner = NerLearner(model, data,
                     best_model_path="/datadrive/models/atis/base_line_final.cpt",
                     lr=0.01, clip=1.0, sup_labels=data.id2label[5:],
                     t_total=num_epochs * len(data.train_dl))

INFO:root:Use lr OneCycleScheduler...


In [ ]:
learner.fit(num_epochs, target_metric='prec')

### Get best results

In [34]:
learner.load_model()

#### Get span results for valid ds (where train support > 3)

In [21]:
import pandas as pd
sup_slots = list(pd.read_csv("/datadrive/JointSLU/data/sup_slots.csv").sup_slots)

In [24]:
from modules.data.data import get_bert_data_loader_for_predict

In [26]:
dl = get_bert_data_loader_for_predict(data_path + "valid_filtered.csv", learner)

In [371]:
span_df = span_df[[s in sup_slots for s in list(span_df.slots)]]

In [27]:
preds = learner.predict(dl)

In [22]:
ss = ["B_"+s for s in sup_slots] + ["I_"+s for s in sup_slots]

In [38]:
from modules.train.train import validate_step


rep = validate_step(learner.data.valid_dl, learner.model, learner.data.id2label, learner.sup_labels)

Mean IOB precision

In [50]:
import numpy as np


np.mean([float(line.split()[1]) for line in rep.split("\n")[2:-5] if int(line.split()[-1]) > 0 and line.split()[0] in ss])

0.8524716981132077

In [52]:
from modules.utils.plot_metrics import get_bert_span_report


clf_report = get_bert_span_report(dl, preds, list(set(learner.data.id2label) - set(ss)))
print(clf_report)

                            precision    recall  f1-score   support

                         O      0.993     0.992     0.993      5495
          arrive-time.time      0.865     0.941     0.901        34
               flight-stop      1.000     1.000     1.000        21
                  day-name      1.000     0.500     0.667         2
                 days-code      0.000     0.000     0.000         1
                state-name      0.000     0.000     0.000         7
        fromloc.state-name      1.000     1.000     1.000        17
            transport-type      1.000     1.000     1.000        10
                 city-name      0.971     0.579     0.725        57
 arrive-date.date-relative      0.500     0.500     0.500         2
        fromloc.state-code      0.920     1.000     0.958        23
 arrive-time.time-relative      0.933     0.903     0.918        31
      return-date.day-name      0.000     0.000     0.000         2
          restriction-code      1.000     1.000

### Get mean and stdv on 10 runs

In [ ]:
from modules.utils.plot_metrics import *
from modules import NerLearner


num_runs = 10
best_reports = []
for i in range(num_runs):
    model = BertBiLSTMAttnCRF.create(len(data.label2idx), bert_config_file, init_checkpoint_pt, enc_hidden_dim=256)
    best_model_path = "/datadrive/models/atis/exp_{}_attn_cased.cpt".format(i)
    learner = NerLearner(model, data,
                         best_model_path=best_model_path, verbose=False,
                         base_lr=0.0001, lr_max=0.001, clip=5.0, use_lr_scheduler=True, sup_labels=data.id2label[5:])
    learner.fit(100, target_metric='prec')
    idx, res = get_mean_max_metric(learner.history, "f1", True)
    best_reports.append(learner.history[idx])

In [32]:
def get_mean_max_metric_(rep, metric_):
    idx = 0
    if metric_ == "rec":
        idx = 1
    elif metric_ == "f1":
        idx = 2
    idx += 1
    return np.mean([float(line.split()[idx]) for line in rep.split("\n")[2:-5] if int(line.split()[-1]) > 0 and line.split()[0] in ss])

In [33]:
import numpy as np

In [34]:
np.mean([get_mean_max_metric_(r, "f1") for r in best_reports]), np.round(np.std([get_mean_max_metric_(r, "f1") for r in best_reports]), 3)

(0.8822570175438595, 0.02)

Best

In [35]:
np.max([get_mean_max_metric_(r, "f1") for r in best_reports])

0.8956666666666666

#### precision

Mean and std

In [36]:
np.mean([get_mean_max_metric_(r, "prec") for r in best_reports]), np.round(np.std([get_mean_max_metric_(r, "prec") for r in best_reports]), 3)

(0.9077385964912281, 0.021)

Best

In [38]:
np.max([get_mean_max_metric_(r, "prec") for r in best_reports])

0.9281929824561402